### Continuação Deploy

Será feito um deploy para uso direto utilizando o streamlit. 

In [ ]:
import pandas as pd
import streamlit as st
import joblib

# Separando as características em numéricas, booleanas (sim ou não) e categóricas:

# Numéricas (inicializando todas com 0)
x_numericos = {'latitude': 0, 'longitude': 0, 'accommodates': 0, 'bathrooms': 0, 'bedrooms': 0, 'beds': 0, 'extra_people': 0, 
               'minimum_nights': 0, 'ano': 0, 'mes': 0, 'n_amenities': 0, 'host_listings_count': 0}

# Booleanas (inicializando todas com 0 (não))
x_tf = {'host_is_superhost': 0, 'instant_bookable': 0}

# Categóricas
x_listas = {'property_type': ['Apartment', 'Bed and breakfast', 'Condominium', 'Guest suite', 'Guesthouse', 
                              'Hostel', 'House', 'Loft', 'Outros', 'Serviced apartment'], 
            'room_type': ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
           'cancellation_policy': ['flexible', 'moderate', 'strict', 'strict_14_with_grace_period']}


dicionario = {}
for item in x_listas:
    # item -> a chave do dicionário
    # x_listas[item] -> é o valor para a chave 'item'
    for valor in x_listas[item]:
        dicionario[f'{item}_{valor}'] = 0


for item in x_numericos:
    # Configurando o formato dos valores 'latitude' e 'longitude' com 5 casas decimais:
    if item == 'latitude' or item == 'longitude':
        valor = st.number_input(f'{item}', step=0.00001, value=0.0, format='%.5f')
    # Configurando o formato dos valores 'extra_people' com 2 casas decimais:    
    elif item == 'extra_people':
        valor = st.number_input(f'{item}', step=0.01, value=0.0) # por padrão, o formato já é com duas casas decimais, por isso não precisa do 'format='%.2f''
    # Configurando o formato do restante das categorias como sendo número inteiro:
    else:
        valor = st.number_input(f'{item}', step=1, value=0) # como o valor de step é 1, o format fica como inteiro por padrão
    # preenchendo o dicionário x_numericos com os valores inseridos:
    x_numericos[item] = valor
    

for item in x_tf:
    # Criando o campo de seleção com os valores 'Sim' e 'Não'
    valor = st.selectbox(f'{item}', ('Sim', 'Não'))
    # preenchendo o dicionário x_tf com os valores inseridos:
    if valor == 'Sim':
        x_tf[item] = 1
    else:
        x_tf[item] = 0


for item in x_listas:
    # Criando campo de seleção com as opções disponíveis para cada característica
    valor = st.selectbox(f'{item}', x_listas[item])
    # Adicionando as características categóricas inseridas no dicionário vazio criado:
    dicionario[f'{item}_{valor}'] = 1

# Criar botão para prever o valor do imóvel
botao = st.button('Prever Valor do Imóvel')

# Adicionando a função de prever o valor do imóvel ao botão criado, com base nos valores inseridos
if botao:
    # Adicionando os valores numéricos e booleanos no dicionário criado
    dicionario.update(x_numericos)
    dicionario.update(x_tf)
    # Transformando num dataframe
    valores_x = pd.DataFrame(dicionario, index=[0])

    # Lendo os arquivos de dados e extraindo o nome das colunas
    dados = pd.read_csv('dados.csv')
    colunas = list(dados.columns)[1:-1]

    # Colocando os valores inseridos pelo usuário no modelo, realizando a previsão e imprimindo o resultado
    valores_x = valores_x[colunas]
    modelo = joblib.load('modelo.joblib')
    preco = modelo.predict(valores_x)
    st.write(preco)